#### UNData API Exercise

In this exercise, you'll redo the data gathering phase of the UNData Exploration project by using APIs instead of downloading csv files.

You'll make use of the [World Bank Indicators API](https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-about-the-indicators-api-documentation). Note that this API does not require an API key. Before attempting the exercise, it would be a good idea to skim through the Documentation page and to check out the [Basic Call Structure article](https://datahelpdesk.worldbank.org/knowledgebase/articles/898581).

In [1]:
import requests
import matplotlib.pyplot as plt
import json
import pandas as pd

### 1. Use the API to get all available data for the _GDP per capita, PPP (constant 2017 international $)_ indicator. Hint: this indicator has code "NY.GDP.PCAP.PP.KD". Adjust the query parameters so that you can retrieve all available rows. Convert the results to a DataFrame.

In [2]:
endpoint = 'https://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.PP.KD?' #add the json format 

params= {
    'Format': 'json',
    'Per_page': '20000', 
}

response = requests.get(endpoint, params=params)
assert response.status_code == 200 #Create a assert statement that the reponse [200] and that it is valid
response.raise_for_status #similar function to get a assert 

<bound method Response.raise_for_status of <Response [200]>>

In [3]:
# endpoint = 'https://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.PP.KD?' #add the json format 

# params= {
#     'Format': 'json',
#     'Per_page': '20000', 
# }

# response = requests.get(endpoint, params=params)
# res = response.json()

# assert response.status_code == 200 #Create a assert statement that the reponse [200] and that it is valid
# response.raise_for_status #similar function to get a assert 

# len(res) #contains 2 lists
# list_data= res[1]
# len(list_data) #A list containing 50 dictionaries

# #normalize the dataframe 
# gdp_ppp_data= pd.DataFrame(list_data)
# gdp_ppp_data.head()

# #clean up the dataframe 
# clean_gdp_data= pd.json_normalize(res[1]) #pull out the nested dictionary info 
# clean_gdp_data= clean_gdp_data.explode('indicator.value')
# clean_gdp_data['indicator.value'].unique()
# clean_gdp_data= clean_gdp_data.rename(columns={'value': 'GDP per capita'})

# clean_gdp_data.head()

In [4]:
def get_UN_api_data(endpoint):
    params= {
        'Format': 'json',
        'Per_page': '20000', 
    }
    response = requests.get(endpoint, params=params)
    assert response.status_code == 200 #Create a assert statement that the reponse [200] and that it is valid
    response.raise_for_status #similar function to get a assert 
    
    #normalize the dataframe 
    output_data= pd.json_normalize(res[1])
    return output_data

In [5]:
un_gdp_data= get_UN_api_data('https://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.PP.KD?')

clean_gdp_data= un_gdp_data.rename(columns={'value': 'GDP per capita'})
clean_gdp_data.head(10)

NameError: name 'res' is not defined

### 2. Now, use the API to get all available data for _Life expectancy at birth, total (years)_. This indicator has code "SP.DYN.LE00.IN". Again, convert the results to a DataFrame.

In [ ]:
# endpoint = 'https://api.worldbank.org/v2/country/all/indicator/SP.DYN.LE00.IN' #add the json format 

# params= {
#     'Format': 'json',
#     'Per_page': '20000', 
# }

# response = requests.get(endpoint, params=params)
# print(response)
# #assert str(print(response)) str.contains(200) #Create a assert statement that the reponse [200] and that it is valid
# res = response.json()
# len(res) #contains 2 lists

# list_data= res[1]
# len(list_data) #A list containing 50 dictionaries
# le_data= pd.DataFrame(list_data)

# #clean up the dataframe 
# le_data= pd.json_normalize(list_data) #pull out the nested dictionary info 
# #clean_le_data= le_data.explode('indicator.value')
# clean_le_data['indicator.value'].unique()
# clean_le_data= clean_le_data.rename(columns={'value': 'life expectancy'})

# clean_le_data.head()

un_gdp_data= get_UN_api_data('https://api.worldbank.org/v2/country/all/indicator/SP.DYN.LE00.IN')

clean_le_data= un_gdp_data.rename(columns={'value': 'life expectancy'})
clean_le_data.head(10)

### 3. Merge the two results DataFrames together. You may want to rename or drop columns prior to merging.

In [ ]:
le_formerge= clean_le_data[['date', 'life expectancy', 'country.value']]
le_formerge.columns= ['year', 'life expectancy', 'country']
le_formerge.head()

In [ ]:
gdp_formerge= clean_gdp_data[['date', 'GDP per capita', 'country.value']]
gdp_formerge.columns= ['year', 'GDP per capita', 'country']
gdp_formerge.head()

In [ ]:
gdp_le_data= pd.merge(le_formerge, gdp_formerge, on=['year', 'country'], how='inner') 
gdp_le_data.head()

### 4. You can also get more information about the available countries (region, capital city, income level classification, etc.) by using the [Country API](https://datahelpdesk.worldbank.org/knowledgebase/articles/898590-country-api-queries). Use this API to pull in all available data. Merge this with your other datasets. Use this to now remove the rows that correspond to regions and not countries.

In [ ]:
endpoint=  'https://api.worldbank.org/v2/country'
params= {
    'Format': 'json',
    'Per_page': '20000' 
}

response = requests.get(endpoint, params=params)
print(response)
res = response.json()
country_region= pd.json_normalize(res[1])
country_region['region.id'].unique() #figure out which column is the correct one to filter
country_region_clean= country_region[country_region['region.id'] != 'NA']
country_region_clean['region.value'].unique() #verifying that the dropped values occured correctly 
country_region_clean.head()
# ##### run it using the function 
# country_region= get_UN_api_data('https://api.worldbank.org/v2/country')
# country_region

In [ ]:
country_region_clean= (country_region_clean
    .rename(columns={'name':'country'})
)
cr_formerge= country_region_clean[['country', 'capitalCity', 'region.value', 'incomeLevel.value']]

In [ ]:
gdp_le_noregion= pd.merge(
    gdp_le_data, 
    cr_formerge, 
    on='country',
    how='inner', 
    validate='many_to_one'
).rename(
    columns={'region.value': 'region', 'incomeLevel.value':'income level'}
)

gdp_le_noregion

In [ ]:
#drop rows with nan values 
gdp_le_noregion= gdp_le_noregion.dropna(subset=['life expectancy', 'GDP per capita'])
gdp_le_noregion

-------------

**Bonus Questions:** After doing a basic request to get all records, you can attempt these exercises to get additional practice using query parameters.

1. Adjust your request so that it returns data just for the United States.

2. Adjust your request so that it returns data just for the United States for the year 2021.

3. Adjust your request so that it returns data just for the United States for the years 2000 through 2021.

4. Adjust your request so that it returns data for the United States and Canada for the years 2000 through 2021.

5. If you haven't already done so and you would like to get some additional practice using loops, use the page parameter in order to pull all records. Do not change the value of the per_page parameter. You will likely need to utilize a loop of some kind in order to pull all records.

6. The endpoint "http://api.worldbank.org/v2/indicator" will return information about all available indicators. See [Indicator API Queries](https://datahelpdesk.worldbank.org/knowledgebase/articles/898599-indicator-api-queries). Use this endpoint to find the code for the "Public Expenditure on Education (% GDP)" indicator.